In [4]:
import re
from collections import OrderedDict

def extract_variables(formula):
    # Шаг 1: Найти все вызовы функций вида "func(...)"
    func_pattern = r'\b([a-zA-Z_]\w*)\s*\('
    functions = set(re.findall(func_pattern, formula))
    
    # Шаг 2: Найти все идентификаторы (переменные и функции)
    var_pattern = r'\b[a-zA-Z_]\w*\b'
    all_identifiers = re.findall(var_pattern, formula)
    
    # Шаг 3: Удалить функции и числа, сохранить порядок
    variables = OrderedDict()
    for identifier in all_identifiers:
        if identifier not in functions and not identifier[0].isdigit():
            variables[identifier] = None
    
    # Шаг 4: Вернуть уникальные переменные в порядке появления
    return list(variables.keys())


['x_1', 'X2', 'x_2', 'a']
['alpha', 'beta_3', 'gamma']
['velocity', 'np', 'time_interval']


In [57]:
import re
from collections import OrderedDict

def extract_variables(formula):
    # Шаг 1: Найти все вызовы функций (включая с точками)
    func_pattern = r'\b([a-zA-Z_][\w.]*)\s*\('
    functions = re.findall(func_pattern, formula)
    
    # Шаг 2: Разбить имена функций на части (например, np.exp → ['np', 'exp'])
    function_parts = set()
    for func in functions:
        parts = func.split('.')
        function_parts.update(parts)
    
    # Шаг 3: Найти все идентификаторы (переменные и функции)
    var_pattern = r'\b[a-zA-Z_]\w*\b'
    all_identifiers = re.findall(var_pattern, formula)
    
    # Шаг 4: Фильтрация
    variables = OrderedDict()
    for ident in all_identifiers:
        # Игнорировать части функций, числа и полные имена функций
        if (
            ident not in function_parts
            and not ident[0].isdigit()
        ):
            variables[ident] = None
    
    return list(variables.keys())



In [2]:
import sympy as sp
import pandas as pd
from IPython.display import display, Math
import numpy as np

df = pd.DataFrame({})

# Разделение формулы на левую и правую части

# Ввод формулы
succesfull_input = True

while succesfull_input:
    x = input("Введите формулу, для которой хотите рассчитать погрешность: ")
    main, str_formula = x.split("=")
    expr_f = f"Вы\\ ввели\\ такую\\ формулу?\\qquad {main} = {sp.latex(sp.sympify(str_formula))}"
    display(Math(expr_f))
    check = input("yes/no ")
    if check == "yes": succesfull_input = False

str_variables = extract_variables(str_formula)

expr = sp.sympify(str_formula)
locals()[main] = expr

variables_float = []
vars_list = []
for x in str_variables:
    display(Math("Введите\\ значение\\ переменной\\quad " + x))
    x_float = float(input())
    variables_float.append(x_float)
    vars_list.append((sp.symbols(x), x_float))
    
df["Переменные"] = [f"${x}$" for x in str_variables]
df["Значения"] = variables_float
display(df.transpose()) 

display(Math("Введите\\ номер\\ переменных\\ с\\ погрешностью\\ через\\ запятую "))

nums = [int(x) for x in input().split(",")]
print(nums)

dvariables_float = np.zeros(len(str_variables), dtype = float)
list_of_all_vars = []
for i in nums:
    display(Math("Введите\\ значение\\ переменной\\quad \\Delta_{" + df.Переменные[i].strip("$") + "}") )
    dvar_float  =(float(input()))
    dvariables_float[i] =  dvar_float
    list_of_all_vars.append((sp.symbols("\\Delta_{" + str_variables[i] + "}")))

df["Погрешность"] = np.zeros(len(str_variables), dtype = float)
list_diffs_symbols = []
for i in nums:
    #list_diffs_symbols = sp.symbols("\\Delta_{" + str_variables[i] + "}")
    df.at[i, "Погрешность"] = dvariables_float[i]
    


symbols = sp.symbols(' '.join(str_variables))
derives = [sp.diff(expr, x) for x in symbols]


df["derives"] = ["$\\frac{\\partial " 
                 + main + "}{\\partial "+ str_variables[i] + "} = " 
                 + sp.latex(derives[i]) + "$" for i in range(len(str_variables))]
list = [(symbols[i])]



#df["derives"] = [] + f"= {sp.latex(sp.diff(expr, x))}
print(df.derives[0])
display(df)

# m = exp(x_1**x_2)

Введите формулу, для которой хотите рассчитать погрешность:  y = x**2


<IPython.core.display.Math object>

yes/no  yes


NameError: name 'extract_variables' is not defined

$\frac{\partial m }{\partial x_1} = \frac{x_{1}^{x_{2}} x_{2} e^{x_{1}^{x_{2}}}}{x_{1}}$